In [ ]:
# Import packages
import pyspark
import dxpy
import dxdata
from pyspark.sql.functions import array_join

In [ ]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [ ]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [ ]:
participant = dataset['participant']

In [ ]:
field_name_dict = {
    'sample_names': 'eid',
    'icd': 'p41270',
}

In [ ]:
field_names = list(field_name_dict.values())

In [ ]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="raw")

In [ ]:
df = df.withColumn("p41270", array_join(df.p41270, '|'))

In [ ]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [ ]:
pandas_df = df.toPandas()

In [ ]:
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})

In [ ]:
start = 0
final = len(pandas_df)
block = 0
proj_dir = f"/phenotype_processing/icd_info/"
nrows = 50000

while start<final:
    end = start + nrows
    table = pandas_df.iloc[start:end, :]
    filename = f"icd_block{block}.csv.gz"
    table.to_csv(filename, index=False)
    upload_file_to_project(filename, proj_dir)
    start += nrows
    block += 1